# Data Convertion

En este archivo se realiza una conversión de datos de JSON y pickle a Parquet y luego a gzip para optimizar la subida de los datos que, en algunos casos pesan demasiado y generarán conflicto a la hora de subirlos a GitHub y dificultarán el procesamiento de los colaboradores.

**Nota:** Este archivo sólo realiza conversión y adaptación de los datos al entorno de trabajo. Si se desea acceder a los DataSets originales empleados para este proyecto, podrán encontrarlos en los siguientes links: [Dataset de Yelp](https://drive.google.com/drive/folders/1TI-SsMnZsNP6t930olEEWbBQdo_yuIZF), [Dataset de Google Maps](https://drive.google.com/drive/folders/1Wf7YkxA0aHI3GpoHc9Nh8_scf5BbD4DA). No olvidar que por el momento sólo se está trabajando con los estados de California, Nueva York y Florida.

In [1]:
# Se importan las librerías necesarias
import pandas as pd
import os
import sys
from datetime import datetime

## Importar los DataSets de Yelp

In [2]:
# Obtener el directorio de trabajo actual
current_dir = os.getcwd()

# Navegar hacia el directorio raíz del proyecto
project_root = os.path.abspath(os.path.join(current_dir, '../..'))

# Agregar la ruta del proyecto al sys.path
sys.path.append(project_root)

In [4]:
# se importan los datasets de yelp del directorio data/raw/yelp/org
# business.pkl (formato pickle)
yelp_business = pd.read_pickle(os.path.join(project_root, 'data/raw/yelp/org/business.pkl'))

# visualizar
yelp_business.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# se importan los datasets de yelp del directorio data/raw/yelp/org
# checkin.json
yelp_checkin = pd.read_json(os.path.join(project_root, 'data/raw/yelp/org/checkin.json'), lines=True)

# visualizar primeros elementos
yelp_checkin.head(3)

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"


In [6]:
# se importan los datasets de yelp del directorio data/raw/yelp/org
# tip.json
yelp_tip = pd.read_json(os.path.join(project_root, 'data/raw/yelp/org/tip.json'), lines=True)

# visualizar primeros elementos
yelp_tip.head(3)

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0


Se ha realizado el pasaje a DF de los datasets con menor volumen de datos a DF en los datasets correspomdientes a Yelp. Restan dos que tendrán que utilizar tecnologías diferentes. 

### Datasets demasiado grandes para trabajarlos con Pandas

Los siguientes archivos son demasiado grandes para procesarlos de forma normal:
- review.json
- user.parquet

Para optimizar la subida de los datos que, en algunos casos pesan demasiado y generarán conflicto a la hora de subirlos a GitHub y dificultarán el procesamiento de los colaboradores, tendrá que considerarse la posibilidad de procesarlos directamente en Linux con herramientas de BigData.

### Columnas Duplicadas
Las columnas duplicadas no permiten la conversión del archivo, por lo que eliminaremos las columnas vacías duplicadas.

In [7]:
# Verificar columnas duplicadas
duplicated_columns = yelp_business.columns[yelp_business.columns.duplicated()].tolist()
print(f"Columnas duplicadas: {duplicated_columns}")

# Eliminar columnas duplicadas
# Esto conserva solo la primera aparición de cada columna y elimina las demás
yelp_business = yelp_business.loc[:, ~yelp_business.columns.duplicated()]

# visualizar los primeros elementos
yelp_business.head(3)

Columnas duplicadas: ['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours']


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."


### Exportar datos 

In [10]:
# se crea una ruta para acceder a los repositorios finales
raw_path = os.path.join(project_root, 'data/raw')
# se exportan los datos de Yelp en formato parquet
yelp_business.to_parquet(f'{raw_path}/yelp/business.parquet', index=False)
yelp_checkin.to_parquet(f'{raw_path}/yelp/checkin.parquet', index=False, compression='gzip')
yelp_tip.to_parquet(f'{raw_path}/yelp/tip.parquet', index=False)